# Flask 

Flask is a Python micro web framework created in 2010 (used by Pinterest and LinkedIn).  
The main difference with bigger web frameworks like Django is that its core is very minimalist : it does not come pre-packaged with a lot of functionalities, it does not require any other library, and allows granularity by importing only extensions for the features we need.

Flask has a lot of extensions offering various features such as validation, upload handling, authentication, database support...

### Installation

- Download the flask framework : `pip install flask`
- Ensure it is correctly installed : `flask --version`

### Hello World

To run a Flask webapp, we just need to create an instance of `Flask` and run it.  
To add routes to the webapp, we use the `app.route()` decorator.

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
    return 'Hello world'

if __name__ == '__main__':
    app.run(debug=True)

From a terminal, run it with `python app.py`, it should start the web server on port 5000 : http://127.0.0.1:5000/  
The `debug` mode allows to reload automatically the web server on code change.

### Path parameters

We can include a path parameter in the path of the decorator.  
The parameter name and type must be specified.  
The path parameter becomes available as a function parameter in the handler function.

In [ ]:
@app.route('/users/<string:user_name>/posts/<int:post_id>')
def get_post(user_name, post_id):
    return 'Fetching post ID %d for user %s' % (post_id, user_name)

### Restrict methods

We can restrict which HTTP methods are allowed for a given route :

In [ ]:
from flask import requests

@app.route('/users', methods=['GET', 'POST'])
def get_users():
    return 'Getting users with method %s' % request.method

The method used can be accessed via the `request` variable provided in the `flask` module.

### Templates

Instead of returning a string to the browser, Flask can use a template file with the `render_template` import.  
Create a `templates` sub-folder (this name is mandatory) and create a template in it.

In [ ]:
@app.route('/index')
def index_template():
    return render_template('index.html')

The Flask templates use the Jinja web templating engine.  
A template can inherit from another one, and contain some blocks that children templates can customize.  

Some common Jinja tags are :
- `{% block xxx %} ... {% endblock %}` to create a block that a child template can populate.
- `{% for x in my_list xxx %} ... {% endfor %}` to loop on all items of a Python list.
- `{% if condition %} ... {% else %} ... {% endif %}` to conditionally add some content in the template.


For example a `base.html` template for HTML pages could be :

In [ ]:
<!DOCTYPE HTML>
<html>
<head>
    <meta charset="utf-8" />
    <title> {% block title %} {% endblock %} </title>
</head>
<body>
    {% block body %} {% endblock %}
</body>
</html>

And a child template could extend this base template and populate the `title` and `body` blocks :

In [ ]:
{% extends 'base.html' %}

{% block title %}
Home
{% endblock %}

{% block body %}
<h1>Home page</h1>
{% endblock %}

Templates can receive parameters from the Python code.  
We can send a list of dict, and then loop on this list in the template with the `{% for x in my_list %}` Jinja tag.  
Some Python code can also be included in the template with the `{{ python code }}` notation.

In [ ]:
{% block body %}
<h2>Friends list :</h2>
<ul>
{% for friend in friends %}
    <li> {{ friend.name }} 
        {% if friend.age %}({{friend.age}} years old) {% endif %}
        who likes {{ friend.hobby }} </li>
{% endfor %}
</ul>
{% endblock %}

### Static files

Flask can serve static content, it needs to be added under a `static/` subfolder.  
This can for example contain subfolders for css, javascript, image files...

Files under this `static` folder can be referenced from a template, for example :

In [ ]:
<link rel="stylesheet" href="static/css/app.css" />

The recommended way is to use the `url_for()` Flask method instead, that allows runtime parameters :

In [ ]:
<link rel="stylesheet" href="{{ url_for('static', filename='css/app.css') }}" />